# <center>Lesson 3: on-disk fields and basic data containers</center>
### <center>yt user/developer workshop, July 2025</center>

# Covered in this lesson:
* what data are present?
* geometric data containers
* querying field data
* unitful arrays and quantities
* field aliases
* profiles

## Previous concepts:
* **dataset**: a collection of data that we load at once. This holds metadata and is cheap to load.
* **frontend**: how we define one distinct source of data. This gives us the **index**, the object that understands the relationship between the spatial layout of the data and the file layout.

### We have not yet accessed any actual data (other than metadata).

## New concepts:
* **field**: an array of values describing a quantity associated with each element in the `dataset`. This is the data we want. Examples: the gas densities of the grid cells, the positions of the particles, the brightness of the pixels.
* **data container**: an object containing one or more elements of a `dataset`. It provides access to `fields` for all the elements it contains.

## Load a dataset

In [ ]:
import yt

ds = yt.load("/Users/britton/EnzoRuns/yt-workshop-2025/primordial_star/DD0157/DD0157")

## On-Disk Fields

These are the fields saved within the dataset. Their names are defined by the frontend.

### What fields are present?

The `field_list` will find this out from the `dataset`.

In [ ]:
ds.field_list

### Field naming convention: all fields are named by a 2-element tuple of `(field type, field name)`

### Notes about creating the `field_list`
* This is the first expensive operation. The `index` gets built.
* The available fields are being detected by inspecting the data.
* The field names are defined by the dataset's `frontend` (mostly).

### Use `field_info` to get more information on each field.

In [ ]:
ds.field_info["enzo", "Density"]

In [ ]:
ds.field_info["enzo", "x-velocity"]

In [ ]:
ds.field_info["enzo", "GasEnergy"]

In [ ]:
ds.field_info["io", "particle_mass"]

### `ds.fields` is an even better way to do this.
* `ds.fields.<field type>.<field name>`
* can use `dir` or tab-completion
  * `dir(ds.fields)` for available types
  * `dir(ds.fields.<field type>` for all fields of this type

In [ ]:
ds.fields.nbody.particle_mass

### Or try this:

In [ ]:
# Doesn't work in presentation mode.
# display(ds.fields)

## Data Containers

Data containers (aka "data objects") are objects that provide access to field data.

yt Documentation: [all available data containers](https://yt-project.org/docs/dev/analyzing/objects.html#available-objects)



### Geometric data containers
* For 3D shapes: they provide access to elements contained within them. All grid cells in which the cell center is within the boundary are considered contained.
* For 0D, 1D, 2D: they access intersecting elements or elements that contain them.

### Some 3D data containers:
* [sphere](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.spheroids.html#yt.data_objects.selection_objects.spheroids.YTSphere)
* [disk](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.disk.html#yt.data_objects.selection_objects.disk.YTDisk)
* [region](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.region.html#yt.data_objects.selection_objects.region.YTRegion) (rectangular prism)
* [ellipsoid](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.spheroids.html#yt.data_objects.selection_objects.spheroids.YTEllipsoid)
* [all data](https://yt-project.org/docs/dev/reference/api/yt.data_objects.static_output.html#yt.data_objects.static_output.Dataset.all_data) (shortcut for `region` with domain boundaries for corners)

### Some 0/1/2D data containers:
* [point](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.point.html#yt.data_objects.selection_objects.point.YTPoint)
* [ray](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.point.html#yt.data_objects.selection_objects.point.YTPoint) (line with start/end point)
* [cutting-plane](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.slices.html#yt.data_objects.selection_objects.slices.YTCuttingPlane) (2D plane or "slice" with arbitrary normal vector)

## Creating data containers

In [ ]:
center = ds.domain_center
radius = 0.1 * ds.domain_width[0]

In [ ]:
sp = ds.sphere(center, radius)

### Data access: query a data container like a dictionary with the full name of the field

In [ ]:
sp["enzo", "Density"]

### What is returned from a query?
* a `unyt` array: a NumPy array with symbolic units that are aware of the dataset's internal units
* an ordered 1D array: the order of elements (grid cells/particles) is always the same. This allows meaningful array operations.
* Data read from disk the first time and grabbed from cache after (`clear_data` deletes the cache).

In [ ]:
sp["enzo", "Density"].to("g/cm**3")

In [ ]:
sp["io", "particle_mass"].to("Msun")

In [ ]:
sp["enzo", "Density"] * sp["enzo", "Temperature"]

In [ ]:
sp["enzo", "Density"]

In [ ]:
sp.clear_data()
sp["enzo", "Density"]

## Position data for grid cells

The "index" field type defines position fields in several references frames especially relevant to some data containers.
* "x[y,z]"
* "spherical_radius[theta,phi]"
* "cylindrical_radius[theta,z]"
These are technically "derived fields" (covered later in Lesson 5).

In [ ]:
sp["index", "x"]

In [ ]:
sp["index", "spherical_radius"]

## More on unitful arrays and quantities

These are of type `unyt_array` and `unyt_quantity` from the [unyt](https://unyt.readthedocs.io/en/stable/) package (spun off from yt).

In [ ]:
from unyt import unyt_array, unyt_quantity

In [ ]:
some_array = unyt_array([1., 2., 3.], "kg")
some_quantity = unyt_array(50000, "km/hr**2")

In [ ]:
(some_array * some_quantity).to("N")

### Converting units
* `.to` returns a copy in the new unit system
* `.convert_to_units` does in-place unit conversion (no data copying)

In [ ]:
some_array.to("Msun")

In [ ]:
some_array

In [ ]:
some_array.convert_to_units("Msun")

In [ ]:
some_array

### Caveat: general `unyt` objects don't know about the dataset's internal units 

In [ ]:
# this won't work
# some_array.to("code_mass")

### Instead, use `ds.arr` and `ds.quan`.

In [ ]:
another_array = ds.arr([1., 2., 3.], "kg")
another_array.to("code_mass")

### Caveat for cosmology simulations: comoving and proper reference frames

Generally speaking, lengths are in the proper frame and appending "cm" returns them in the comoving frame.

In [ ]:
print (ds.domain_width.to("Mpc"))
print (ds.domain_width.to("Mpccm"))
print (ds.domain_width.to("Mpccm/h"))

Take great care when working with multiple cosmological snapshots at once. For length-related quantities, use "unitary" as it is known to always be a constant. This is a unit system normalized to the size of the box.

In [ ]:
print (ds.domain_width.to("unitary"))

## Field aliases
* Field types and names exist for quantities that many data formats have in common.
* Fields are all named using the lowercase-underscore convention.
* Data is returned in universally defined units (i.e., not frontend-specific).

The best example of this is **"gas"**.

In [ ]:
sp["gas", "density"]

The "gas" field type also defines several other common "derived fields" (Lesson 5).

## Useful helper functions
* These are associated with the `dataset` object (**not** with the data containers).
* They will run on the entire dataset and so should not be expected to be fast.
* They can be used to make informed decisions about defining your data containers. (For example, center a sphere on the point of maximum density.)
* [find_max](https://yt-project.org/docs/dev/reference/api/yt.data_objects.static_output.html#yt.data_objects.static_output.Dataset.find_max)
* [find_min](https://yt-project.org/docs/dev/reference/api/yt.data_objects.static_output.html#yt.data_objects.static_output.Dataset.find_min)
* [field_values_at_point](https://yt-project.org/docs/dev/reference/api/yt.data_objects.static_output.html#yt.data_objects.static_output.Dataset.find_field_values_at_point)
* [field_values_at_points](https://yt-project.org/docs/dev/reference/api/yt.data_objects.static_output.html#yt.data_objects.static_output.Dataset.find_field_values_at_points)

In [ ]:
ds.find_max(("gas", "density"))

## Profiles
* Profiles are essentially multidimensional histograms of field data.
* canned plotting methods exist for 1D (ProfilePlot) and 2D (PhasePlot) profiles (covered in Lesson 6).
* yt supports 1D, 2D, and 3D profiles via the [create_profile](https://yt-project.org/docs/dev/reference/api/yt.data_objects.profiles.html#yt.data_objects.profiles.create_profile) function
* given 1, 2, or 3 "bin fields", one "profile field", and (optionally) a "weight field", the profile yields:
  * unweighted profiles: the sum of the profile field in bins of the bin fields
  * weighted profiles: the weighted average of the profile field (weighted by the weight field) in bins of the bin fields

In [ ]:
value, center = ds.find_max(("gas", "density"))

In [ ]:
sp = ds.sphere(center, radius)

In [ ]:
profile_fields = [("index", "spherical_radius")]
bin_fields = [("gas", "density")]
weight_field = ("gas", "cell_mass")

profile = yt.create_profile(sp, profile_fields, bin_fields, weight_field=weight_field)

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
plt.loglog(profile.x.to("pc"), profile["gas", "density"])
plt.xlabel("r [pc]")
plt.ylabel("$\\rho\\ [g/cm^{3}]$")
plt.show()

### Now make an informed choice about a new sphere.

In [ ]:
sp_small = ds.sphere(center, (5, "pc"))

### Arguments for data containers
Generally speaking, quantities can be expressed as:
* `unyt_arrays` (e.g., ds.quan(5, "pc"))
* tuples of (value, string unit) (e.g., (5, "pc"))
* floats or arrays without units. Some assumption is made, usually some form of code unit (e.g., "code_length").

In [ ]:
profile = yt.create_profile(sp_small, [("gas", "density")], [("gas", "temperature")],
                            weight_field=("gas", "cell_mass"))

In [ ]:
plt.loglog(profile.x, profile["gas", "temperature"])
plt.xlabel("$\\rho\\ [g/cm^{3}]$")
plt.ylabel("T [K]")
plt.show()

### Weighted standard deviations
* for weighted profiles, profile objects have a `standard_deviation` attribute holding the weighted standard deviations

In [ ]:
T_mean = profile["gas", "temperature"]
T_std = profile.standard_deviation["gas", "temperature"]

In [ ]:
plt.loglog(profile.x, T_mean)
plt.fill_between(profile.x, y1=T_mean-T_std, y2=T_mean+T_std, alpha=0.5)
plt.xlabel("$\\rho\\ [g/cm^{3}]$")
plt.ylabel("T [K]")
plt.show()

### Useful profile object attributes
* filtering out empty bins:
  * profile objects have a `used` attribute
  * this is a boolean array indicating non-empty bins
* bin edges vs. bin centers
  * `x[y,z]` attribute gives bin centers (same shape as profile data)
  * `x[y,z]_bins` attributes gives bin edges

In [ ]:
profile.used

In [ ]:
print (profile.x.size, profile.x_bins.size)

### 2D profiles

In [ ]:
bin_fields = [("gas", "density"),
              ("gas", "temperature")]
profile_fields = [("gas", "cell_mass")]

profile = yt.create_profile(sp_small, bin_fields, profile_fields, weight_field=None)

In [ ]:
from yt.utilities.physical_constants import mh
X = profile.x / mh
Y = profile.y
Z = profile["gas", "cell_mass"].to("Msun").T

plt.xscale("log")
plt.yscale("log")
plt.xlabel("$\\rho\\ [1/cm^{3}]$")
plt.ylabel("T [K]")

my_norm = matplotlib.colors.LogNorm(vmin=Z[Z>0].min(), vmax=Z.max())
my_plot = plt.pcolormesh(X, Y, Z, norm=my_norm)
plt.colorbar(my_plot, label="$M_{gas}\\ [M_{\\odot}]$")
plt.show()

## What else can you do with geometric data containers?

Geometric data containers can be the building blocks of more complex structures.

### Combine them!
* operators:
  * `&`: intersection (AND)
  * `|` or `+`: union (OR)
  * `^`: exclusive or (XOR)
  * `-`: subtraction (NEG)
* [intersection](https://yt-project.org/docs/dev/reference/api/yt.data_objects.selection_objects.boolean_operations.html#yt.data_objects.selection_objects.boolean_operations.YTIntersectionContainer3D) and `union`:
  * same as `&`, `|` but faster
  * can construct from multiple objects at once

The circumgalactic medium as a shape?
```
my_cgm = my_halo_sphere - my_galaxy_disk
```